In [1]:
import h5py
import numpy as np
from tqdm.notebook import tqdm
import os
import glob
import matplotlib.pyplot as plt
import time

from train_Informer_mul import *

from utils import *
from models import *

import warnings
warnings.filterwarnings('ignore')

assert torch.cuda.is_available()
device = 'cuda'

In [2]:
train_f = sorted(glob.glob('../stock_price/train/*.hdf'))
train_f.remove('../stock_price/train\\20190425_20190510.hdf')
test_f = sorted(glob.glob('../stock_price/test/*.hdf'))
# test_f.remove('../stock_price/test\IC2003_20191018.hdf') # batch为256时，第71个batch大小为2， 不足以用于预测

In [3]:
len(test_f)

174

In [4]:
train_f = train_f[-4:]
train_f

['../stock_price/train\\20190617_20190628.hdf',
 '../stock_price/train\\20190628_20190710.hdf',
 '../stock_price/train\\20190710_20190723.hdf',
 '../stock_price/train\\20190723_20190731.hdf']

In [5]:
# Hyperparams
enc_in = 15
dec_in = 2
c_out = 1 
seq_len = 20
out_len = 13
d_model = 16
d_ff = 8
n_heads = 6
label_len = 10
e_layers = 3
d_layers = 2
scaler = False
opt_s = False

decay = 1500
dropout = 0.1
batch_size = 256
val_batch = 512
lr = 0.004

In [6]:
Informer = Train_Informer_mul(enc_in, dec_in, c_out, seq_len, out_len, d_model, d_ff, n_heads, 
                                e_layers, d_layers, label_len,
                                dropout, batch_size, val_batch, lr,
                              device, train_f, test_f,scaler, decay, opt_s)

In [7]:
Informer._build_model()

Informer(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(15, 16, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(2, 16, kernel_size=(3,), stride=(1,), padding=(1,), padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0): EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): FullAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (query_projection): Linear(in_features=16, out_features=12, bias=True)
          (key_projection): Linear(in_features=16, out_features=12, bias=True)
          (value_projection): Linear(in_features=16

In [8]:
Informer._selct_optim('adam')
Informer._selct_scheduler(patience=10, factor=0.2)

In [9]:
Informer.lr = 0.0005
Informer.warmup_train(0.0001, 5, f='../stock_price/train/20190723_20190731.hdf')

Learning Rate is set to 0.0001
Warm


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:  1 |Train_Loss:0.503022 |R2:-0.229342|Val_Loss:0.461641 |R2:0.004480 |Rate:0.516 |lr:0.000100
Learning Rate is set to 0.00018
Epoch:  2 |Train_Loss:0.422758 |R2:-0.057839|Val_Loss:0.449862 |R2:0.026769 |Rate:0.555 |lr:0.000180
Learning Rate is set to 0.00026000000000000003
Epoch:  3 |Train_Loss:0.400921 |R2:-0.023439|Val_Loss:0.443734 |R2:0.067144 |Rate:0.594 |lr:0.000260
Learning Rate is set to 0.00034
Epoch:  4 |Train_Loss:0.386362 |R2:-0.003109|Val_Loss:0.437875 |R2:0.069897 |Rate:0.597 |lr:0.000340
Learning Rate is set to 0.00042
Epoch:  5 |Train_Loss:0.375458 |R2:0.015125|Val_Loss:0.431940 |R2:0.086103 |Rate:0.608 |lr:0.000420
Learning Rate is set to 0.0005
Learning Rate is set to 0.0005
Warm Up Done


In [11]:
Informer._set_lr(0.004)

Learning Rate is set to 0.004


In [ ]:
Informer.train(epochs=100, train_all=True, f='../stock_price/train/20190723_20190731.hdf',
             val_all=True, testfile=None, save='train',patience=40)

  0%|          | 0/100 [00:00<?, ?it/s]

Save here
Epoch:  5 |Train_Loss:0.440339 |R2:0.145772|Val_Loss:0.432515 |R2:0.171887 |Rate:0.639 |lr:0.004000
Validation R2 increased (-inf --> 0.171887).  Saving model ...
Save here
Epoch:  6 |Train_Loss:0.414954 |R2:0.252557|Val_Loss:0.442100 |R2:0.158280 |Rate:0.625 |lr:0.004000
EarlyStopping counter: 1 out of 40
